## Import Modules

In [3]:
from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import json
import os

## Initialize Variables

In [7]:
project="robust-habitat-439810-p4"
location="us-central1"
MODEL_NAME = "text-embedding-preview-0815"
sentence_file_path = "lakeside_sentences.json"
index_name = "lakeside_index"

## Initialize Model and Vector Index

In [13]:
aiplatform.init(project=project,location=location)
vertexai.init()
model = GenerativeModel("gemini-pro")
index_id = "6361453220916625408" # Get from Index Endpoints page
lakeside_index_ep = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name=index_id)

## Helper Functions

In [20]:
def generate_text_embeddings(sentences) -> list:    
    model = TextEmbeddingModel.from_pretrained(MODEL_NAME)
    embeddings = model.get_embeddings(sentences)
    vectors = [embedding.values for embedding in embeddings]
    return vectors

def generate_context(ids,data):
    concatenated_names = ''
    for id in ids:
        for entry in data:
            if entry['id'] == id:
                concatenated_names += entry['sentence'] + "\n" 
    return concatenated_names.strip()

def load_file(sentence_file_path):
    data = []
    with open(sentence_file_path, 'r') as file:
        for line in file:
            entry = json.loads(line)
            data.append(entry)
    return data

## Perform Semantic Search

In [24]:
data=load_file(sentence_file_path)

query = ["What are the benefits of wildlife to people"]
qry_emb=generate_text_embeddings(query)

In [29]:
response = lakeside_index_ep.find_neighbors(
    deployed_index_id = index_name,
    queries = [qry_emb[0]],
    num_neighbors = 10
)

In [30]:
response

[[MatchNeighbor(id='bb1d4466-6bc6-4a47-8501-9f9330407276', distance=0.8008383512496948, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[], sparse_embedding_values=[], sparse_embedding_dimensions=[]),
  MatchNeighbor(id='6580a845-8c78-4d76-bec8-273218ce2d5a', distance=0.800221860408783, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[], sparse_embedding_values=[], sparse_embedding_dimensions=[]),
  MatchNeighbor(id='1e8c1b69-84a1-4cd6-b6dc-aeaef52c7fdf', distance=0.7750945091247559, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[], sparse_embedding_values=[], sparse_embedding_dimensions=[]),
  MatchNeighbor(id='14fb7a2f-d12a-4782-b88c-4cc48e3e9adf', distance=0.7558717131614685, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[], sparse_embedding_values=[], sparse_embedding_dimensions=[]),
  MatchNeighbor(id='69421

## Load IDs and Generate Context

In [31]:

matching_ids = [neighbor.id for sublist in response for neighbor in sublist]

context = generate_context(matching_ids,data)
print(f"context = {context}")


context = Apart from a bird feeder in the backyard, we can also take up other active pastimes, such as hiking, hunting, canoeing or wildlife photographing to relieve our parched nerves. Since prehistoric times, animals have been highly useful to us in providing food, clothing and source of income. Benefits to Natural Processes Wildlife plays an essential role in the ecological and biological processes that are yet again significant to life. The normal functioning of the biosphere depends on endless interactions amongst animals, plants, and microorganisms
Importance of Wildlife The wildlife comprises all living organism (plants, animals, microorganisms) in their natural habitats which are neither cultivated or domesticated nor tamed. But in its strictest sense, it includes uncultivated mammals, reptiles, birds and fishes etc. Certain importance of wildlife as below: 1. Ecological Balance: Wildlife maintains balance of nature through‐ (a) Regulation of population of different species. (b

## Inject Context and Invoke Model

In [32]:
prompt=f"Based on the context delimited in backticks, answer the query. ```{context}``` {query}"

chat = model.start_chat(history=[])
response = chat.send_message(prompt)
print(response.text)

The passage mentions various benefits of wildlife to people. Here are some of the key points:

* **Ecological Balance:** Wildlife plays a crucial role in maintaining the ecological balance by regulating populations of different species and ensuring the proper functioning of food chains. This, in turn, supports human life by providing clean air, water, and fertile soil.
* **Economic Value:** Wildlife provides numerous valuable resources, including food, timber, medicines, and tourism revenue. Many industries rely on wildlife for their survival.
* **Scientific Value:** Scientists have discovered important medical products and gained valuable knowledge about various plants and animals through studying wildlife.
* **Aesthetic Value:** Wildlife contributes to the natural beauty of the Earth, which provides recreational and spiritual benefits to humans.
* **Educational Value:** Learning about wildlife can teach us about the natural world and our place in it, fostering a sense of respect and 

In [33]:
prompt=f"Based on the context delimited in backticks, answer the query in a single paragraph of 100 words exactly. ```{context}``` {query}"

chat = model.start_chat(history=[])
response = chat.send_message(prompt)
print(response.text)

Wildlife provides many benefits to people, including food, shelter, and medicine. Wildlife also plays a key role in the ecological and biological processes that are essential for life. For example, animals help to regulate the population of different species, and they are involved in food chains that provide energy to all living things. Additionally, wildlife provides economic benefits to humans through tourism, exports, and the sale of wildlife products. It is important to conserve wildlife for both its intrinsic value and for the benefits that it provides to people.
